In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.marsDB
collection = db.news

# Mars news

In [53]:
news_url = 'https://mars.nasa.gov/news/'
news_html = requests.get(news_url)
news_soup = BeautifulSoup(news_html.text)
news_response = news_soup.find_all('div', class_='image_and_description_container')

In [55]:
news_titles = []
news_p = []
for i in range(len(news_response)):
    news_titles.append(news_response[i].find_all('img', alt=True)[1].get('alt',''))
    news_p.append(news_response[i].find('div',class_='rollover_description_inner').text.strip())

['Why This Martian Full Moon Looks Like Candy', 'NASA Garners 7 Webby Award Nominations', "NASA's Opportunity Rover Mission on Mars Comes to End", "NASA's InSight Places First Instrument on Mars", 'NASA Announces Landing Site for Mars 2020 Rover', 'Opportunity Hunkers Down During Dust Storm']
["For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.", 'Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.', "NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.", 'In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.', 'After a five-year search, NASA has ch

In [43]:
for i in range(len(news_titles)):
    news = {'title':news_titles[i],'p':news_p[i]}
    collection.insert_one(news)

# Mars picture

In [47]:
from splinter import Browser

In [81]:
collection = db.image

In [51]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
picture_html = browser.html
picture_soup = BeautifulSoup(picture_html, 'html.parser')


browser.click_link_by_partial_text('FULL IMAGE')

In [78]:
part_url = picture_soup.find_all('a', class_='button fancybox')[0].get('data-fancybox-href')

In [82]:
image_url = 'https://www.jpl.nasa.gov' + part_url
image = {'image_url':image_url}

In [83]:
collection.insert_one(image)

# Mars weather

In [86]:
collection = db.weather

In [85]:
weather_url =  'https://twitter.com/marswxreport?lang=en'
weather_html = requests.get(weather_url)
weather_soup = BeautifulSoup(weather_html.text)

In [129]:
mars_weather_all = weather_soup.find_all('p', class_='TweetTextSize')

In [136]:
mars_weather = mars_weather_all[1].text.replace ('\n', ', ')

In [138]:
mars_weather = mars_weather[:-26]

'InSight sol 165 (2019-05-15) low -100.3ºC (-148.6ºF) high -18.2ºC (-0.7ºF), winds from the SW at 4.6 m/s (10.4 mph) gusting to 13.7 m/s (30.6 mph), pressure at 7.50 hPa'

In [140]:
mars_weather = {'mars_weather': mars_weather}

In [141]:
collection.insert_one(mars_weather)

# Mars facts

In [150]:
collection = db.facts

In [143]:
facts_url = 'https://space-facts.com/mars/'
facts_html = requests.get(facts_url)
facts_soup = BeautifulSoup(facts_html.text)

In [144]:
mars_facts_table = pd.read_html(facts_url)
mars_facts = mars_facts_table[0]
mars_facts = mars_facts.to_html()

In [149]:
mars_facts = {'mars_facts': mars_facts}

In [151]:
collection.insert_one(mars_facts)

# Mars hemispheres

In [154]:
collection = db.hemisphere

In [155]:
hemisphere_image_url = [{'title':'Cerberus hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
{'title': 'Schiaparelli hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
{'title': 'Syrtis major hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
{'title': 'Valles Marineris hemisphere', 'img_url':'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [157]:
hemisphere_img = {'hemis_img': hemisphere_image_url}

In [158]:
collection.insert_one(hemisphere_img)